In [1]:
from src.opensource.llama_frontend import chat_pipeline
from src.opensource.templates import SYS_LLAMA_TEMPLATE, INST_LLAMA_TEMPLATE, SYS_LLAMA_TRANSLATE_TEMPLATE, SYS_BLOOMZ_TEMPLATE, INST_BLOOMZ_TEMPLATE

/anaconda/envs/culture/lib/python3.8/site-packages/transformers/models/auto/configuration_auto.py:995: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/anaconda/envs/culture/lib/python3.8/site-packages/transformers/models/auto/auto_factory.py:460: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/anaconda/envs/culture/lib/python3.8/site-packages/transformers/utils/hub.py:373: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/anaconda/envs/culture/lib/python3.8/site-packages/transformers/models/auto/tokenization_auto.py:631: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [2]:
agent = chat_pipeline(prompt_text=SYS_BLOOMZ_TEMPLATE, partial_template=True)

In [3]:
inst_template = INST_BLOOMZ_TEMPLATE

In [4]:
output_1 = agent.predict(human_input=inst_template)

/anaconda/envs/culture/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [5]:
output_1

'\n\n\n- CLUE 1 - Traditional Saree from West bengal'

In [6]:
output_2 = agent.predict(human_input="Your first guess is not correct")

In [7]:
output_2

'CLUE 2'